In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [3]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 74701a8c35f6...   0% ▕▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕▏ 5.6 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   3% ▕▏  45 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   6% ▕▏  74 MB/1.3 GB                  pulling ma

https://medium.com/data-science/how-to-talk-to-a-pdf-file-without-using-proprietary-models-cli-streamlit-ollama-6c22437ed932

In [4]:
!pip install streamlit PyPDF2 langchain langchain-community spacy faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [5]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import subprocess
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate
import os
import re
import psutil

In [7]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [8]:
def pdf_read(pdf_doc):
    """Read the text from PDF document."""
    text = ""
    for pdf in pdf_doc:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

In [9]:
def create_text_chunks(text, chunk_size=1000, chunk_overlap=200):
    """Create text chunks from a large text block."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    text_chunks = text_splitter.split_text(text)
    return text_chunks

In [10]:
embeddings = SpacyEmbeddings(model_name="en_core_web_sm")

In [11]:
def vector_store(text_chunks):
    """Create a vector store for the text chunks."""
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_db")

In [12]:
def query_llama_via_cli(input_text):
    """Query the Llama model via the CLI."""
    try:
        # Start the interactive process
        process = subprocess.Popen(
            ["ollama", "run", "llama3.1"],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,  # Ensure that communication takes place as text (UTF-8)
            encoding='utf-8',  # Set UTF-8 encoding explicitly
            errors='ignore',  # Ignore incorrect characters
            bufsize=1
        )

        # Send the input to the process
        stdout, stderr = process.communicate(input=f"{input_text}\n", timeout=30)

        # Check error output
        if process.returncode != 0:
            return f"Error in the model request: {stderr.strip()}"

        # Filter response and remove control characters
        response = re.sub(r'\x1b\[.*?m', '', stdout)  # Remove ANSI codes

        # Extract the relevant answer
        return extract_relevant_answer(response)

    except subprocess.TimeoutExpired:
        process.kill()
        return "Timeout for the model request"
    except Exception as e:
        return f"An unexpected error has occurred: {str(e)}"

In [13]:
def extract_relevant_answer(full_response):
    """Extract the relevant response from the full model response."""
    response_lines = full_response.splitlines()

    # Search for the relevant answer; if there is a marker, it can be used here
    if response_lines:
        # Assume that the answer comes as a complete return to be filtered
        return "\n".join(response_lines).strip()

    return "No answer received"

In [14]:
def get_conversational_chain(context, ques):
    """Create the input for the model based on the prompt and context."""
    # Define the prompt behavior
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are an intelligent and helpful assistant. Your goal is to provide the most accurate and detailed answers
                possible to any question you receive. Use all available context to enhance your answers, and explain complex
                concepts in a simple manner. If additional information might help, suggest further areas for exploration. If the
                answer is not available in the provided context, state this clearly and offer related insights when possible.""",
            ),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ]
    )

    # Combine the context and the question
    input_text = f"Prompt: {prompt.format(input=ques)}\nContext: {context}\nQuestion: {ques}"

    # Request to the model
    response = query_llama_via_cli(input_text)
    st.write("PDF: ", response)  # The answer is displayed here

In [15]:
def user_input(user_question, pdf_text):
    """Processes the user input and calls up the model."""
    # Use the entire text of the PDF as context
    context = pdf_text

    # Configure and request
    get_conversational_chain(context, user_question)

In [16]:
def main():
    """Main function of the Streamlit application."""
    st.set_page_config(page_title="CHAT WITH YOUR PDF")
    st.header("PDF CHAT APP")

    pdf_text = ""
    pdf_doc = st.file_uploader("Upload your PDF Files and confirm your question", accept_multiple_files=True)

    if pdf_doc:
        pdf_text = pdf_read(pdf_doc)  # Read the entire PDF text

    user_question = st.text_input("Ask a Question from the PDF Files")

    if user_question and pdf_text:
        user_input(user_question, pdf_text)

    # Monitor RAM consumption
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 ** 2)  # Conversion to megabytes
    st.sidebar.write(f"Memory usage: {memory_usage:.2f} MB")

if __name__ == "__main__":
    main()

2025-04-12 00:53:28.274 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-12 00:53:28.277 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-12 00:53:28.345 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-12 00:53:28.346 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-12 00:53:28.347 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-12 00:53:28.348 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-12 00:53:28.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [ ]:
!pip install pyngrok




from pyngrok import ngrok
import threading
import time

ngrok_token = "x"
ngrok.set_auth_token(ngrok_token)

# Start ngrok in a separate thread to avoid blocking
def start_ngrok():
    public_url = ngrok.connect(3000).public_url
    print(f"🚀 Ngrok Tunnel Open: {public_url}")

ngrok_thread = threading.Thread(target=start_ngrok)
ngrok_thread.start()

# Wait for ngrok to start (optional)
time.sleep(5)

# Execute your node.js script




!streamlit run pca1.py

In [ ]:
!pip install pyngrok

In [1]:
!nohup ollama serve &
from pyngrok import ngrok

nohup: appending output to 'nohup.out'


In [2]:
ngrok.set_auth_token("x")

In [ ]:

public_url = ngrok.connect(8501).public_url
print(f"🚀 Ngrok Tunnel Open: {public_url}")
!streamlit run pca1.py

In [ ]:
streamlit run your_app_name.py --server.fileWatcherType none

المشكلة الرئيسية هي عدم توافق بين كيفية عمل مراقب الملفات في Streamlit (الذي يفترض بنية وحدة بايثون قياسية) والتنفيذ الداخلي لـ torch.classes في PyTorch. يحاول المراقب فحص شيء في PyTorch بطريقة غير مدعومة، مما يتسبب في انهيار PyTorch، وهذا الانهيار بدوره يسبب مشكلة ثانوية في نظام معالجة الأخطاء أو نظام asyncio الخاص بـ Streamlit.
الحل المقترح:
الحل الأكثر شيوعًا لهذه المشكلة هو تعطيل مراقب الملفات في Streamlit عند تشغيل التطبيق. يمكنك القيام بذلك عن طريق إضافة الخيار --server.fileWatcherType none إلى أمر التشغيل:
streamlit run your_app_name.py --server.fileWatcherType none
Use code with caution.
Bash
استبدل your_app_name.py باسم ملف بايثون الخاص بتطبيقك.
سيؤدي هذا إلى منع Streamlit من محاولة فحص ملفات مكتبة torch بحثًا عن تغييرات، وبالتالي تجنب الخطأ. الجانب السلبي الوحيد هو أن Streamlit لن يعيد تحميل التطبيق تلقائيًا إذا قمت بتعديل الشيفرة المصدرية أثناء تشغيله؛ ستحتاج إلى إيقافه وإعادة تشغيله يدويًا لرؤية التغييرات.

In [4]:
!nohup ollama serve &
from pyngrok import ngrok

nohup: appending output to 'nohup.out'


In [5]:
ngrok.set_auth_token("x")

In [ ]:

public_url = ngrok.connect(8501).public_url
print(f"🚀 Ngrok Tunnel Open: {public_url}")
!streamlit run pca1.py --server.fileWatcherType none

🚀 Ngrok Tunnel Open: https://211f-35-194-129-50.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.129.50:8501

